<a href="https://colab.research.google.com/github/harvard-visionlab/sroh/blob/main/2022/ipcl_vs_brains_wrsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://osf.io/8qcdp/download -O ObjectOrientationData.mat
!wget -nc https://osf.io/49qeb/download -O InanimateObjectsData.mat
!wget -nc https://osf.io/x9dz4/download -O InanimateObjects.zip
!unzip InanimateObjects.zip
!mkdir -p Stimuli 
!mv InanimateObjects ./Stimuli/InanimateObjects
!wget -c https://raw.githubusercontent.com/harvard-visionlab/sroh/main/2022/feature_extractor.py

In [ ]:
import os
import scipy.io as sio
import numpy as np


def load_brain_data(dataset, brain_regions=['EarlyV', 'pOTC', 'aOTC']):
    assert dataset in ['InanimateObjects', 'ObjectOrientation']
    path = f'{dataset}Data.mat'
    D = sio.loadmat(path, struct_as_record=False, squeeze_me=True)
    rdms = {r: D['rdms'].__dict__[r] for r in brain_regions}
    betas = {r: D['betas'].__dict__[r] for r in brain_regions}
    reliability = {r: D['reliability'].__dict__[r] for r in brain_regions}
    image_names = [f.strip() for f in D['image_names']]
    return rdms, betas, reliability, image_names

In [ ]:
rdms, betas, reliability, image_names = load_brain_data('InanimateObjects')
for k,v in rdms.items(): print(k, v.shape)

In [ ]:
# for k,v in betas.items(): 
#   print(k, v.shape)
#   for brain_subject in v:
#     print(brain_subject.shape)

In [ ]:
# rdms, betas, reliability, image_names = load_brain_data('ObjectOrientation')
# for k,v in rdms.items(): print(k, v.shape)

In [ ]:
import numpy as np
from fastprogress.fastprogress import progress_bar
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import r2_score
from pdb import set_trace
from copy import deepcopy
from tqdm import tqdm
import sklearn

default_alphas = np.concatenate([np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]), np.logspace(1, 5, 50)])

def leave_one_out_ridge(X, y, alphas=default_alphas, fit_intercept=True, normalize=True, mb=None):
    '''
        Construct predicted brain patterns by training on N-1 items, 
        and then predicting the held out item.
        
        X: model responses [numItems x numFeatures]
        y: brain responses [numItems x numVoxels]
    '''
    n_items, n_features = X.shape
    n_voxels = y.shape[1]
    y_pred = np.zeros(y.shape)

    ALPHAS = []
    COEF_M = np.zeros((n_voxels, n_features))
    INTERCEPT = []
    for iter_count, test_idx in enumerate(progress_bar(range(n_items), total=n_items)):
        train_idxs = np.ones(n_items) == True
        train_idxs[test_idx] = False
        test_idxs = ~train_idxs  

        clf = RidgeCV(alphas=alphas, fit_intercept=fit_intercept)

        if normalize:
            scaler = StandardScaler()
            scaler.fit(X[train_idxs])
            X_train = scaler.transform(X[train_idxs])
            X_test = scaler.transform(X[test_idxs])        
        else:
            X_train = X[train_idxs]
            X_test = X[test_idxs]
              
        clf.fit(X_train, y[train_idxs])
        y_pred[test_idxs] = clf.predict(X_test)
        
        ALPHAS.append(clf.alpha_)
        COEF_M += clf.coef_
        INTERCEPT.append(clf.intercept_)

    ALPHAS = np.stack(ALPHAS)
    COEF_M /= iter_count
    INTERCEPT = np.stack(INTERCEPT)
    R2 = r2_score(y, y_pred, multioutput='raw_values')
    
    return {
        "n_items": n_items,
        "n_features": n_features,
        "n_voxels": y.shape[1],
        "ALPHAS": ALPHAS,
        "COEF_M": COEF_M,
        "INTERCEPT": INTERCEPT,
        "R2": R2,
        "y_pred": y_pred
    }

In [ ]:
import torch
from torchvision import models, transforms 
from PIL import Image 
from natsort import natsorted 
from glob import glob 
from pathlib import Path 
from feature_extractor import FeatureExtractor

def prepare_images(dataset='InanimateObjects', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):

    # standard imagenet normalization
    normalize = transforms.Normalize(mean=mean, std=std)

    transform = transforms.Compose([
        lambda x: Image.open(x),    # use PIL to open the image
        transforms.Resize(224),     # resize shorted edge to 224 pixels
        transforms.CenterCrop(224), # center crop if not square
        transforms.ToTensor(),      # convert from RGB (HxWxC) to channels first torch tensor [CxHxW]
        normalize                   # normalize by imagenet stats
    ])
    files = natsorted(glob(f'./Stimuli/{dataset}/*.jpg'))
    file_names = [Path(f).name for f in files] 
    imgs = torch.stack([transform(f) for f in files])

    return imgs

def fit_encoding_model(betas, model_name='alexnet', layer_name='classifier.5',
                       dataset='InanimateObjects', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
          
    print("==> prepare images")
    imgs = prepare_images(dataset=dataset, mean=mean, std=std)
    
    print("==> load pretrained model")
    model = models.__dict__[model_name](pretrained=True)

    print("==> extract activation map for the given layer")
    pred_rdms = {}
    feat_rdms = {}
    model.eval()   # <-- very important, freeze normalization stats, no dropout etc.
    with FeatureExtractor(model, [layer_name]) as extractor:
        features = extractor(imgs)
        for layer_name,feat in features.items():
            # retain spatial information, but flatten rows into a 1D feature vector
            X = torch.flatten(feat, 1)
            feat_rdm = 1 - np.corrcoef(X)
            feat_rdms[layer_name] = feat_rdm
            
            print(f"==> fitting ridge regression model ({layer_name}) (numFeatures={X.shape[1]})")
            results = leave_one_out_ridge(X, betas, fit_intercept=True, normalize=False)
            
            # compute the predicted neural RDM
            pred_rdm = 1 - np.corrcoef(results['y_pred'])
            pred_rdms[layer_name] = pred_rdm
              
            # now do something with the rdms, e.g., save them for our split-half analysis
          
    return pred_rdms, feat_rdms, results

In [ ]:
betas.keys()

In [ ]:
sub_betas = betas['pOTC'][0].transpose()
sub_betas.shape

In [ ]:
pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
results.keys()

In [ ]:
results.keys()

In [ ]:
results['n_voxels'], results['n_features']

In [ ]:
results['y_pred'].shape

In [ ]:
# results['R2']

In [ ]:
import seaborn as sns 

sns.distplot(results['R2'])

In [ ]:
results['COEF_M'].shape

In [ ]:
ys = results['COEF_M'].mean(axis=0)
xs = list(range(len(ys)))
sns.lineplot(xs, ys)

In [ ]:
sns.distplot(results['COEF_M'].mean(axis=0))